In [1]:
from faker import Faker          
import numpy as np
import pandas as pd
import psycopg2 
import psycopg2.extras 
import os
from io import StringIO
import datetime
import random

In [2]:
faker = Faker()
final = pd.DataFrame()

In [12]:
conn = psycopg2.connect(
    database="credit_risk",
    user="rootadmin",
    password="securepassword",
    host="creditrisk.cmcz6iv7alyg.eu-central-1.rds.amazonaws.com",
    port='5432'
)

#Load Data into Clients

In [4]:
fake = Faker() 
def create_clientdata(x): 
  
    # dictionary 
    client ={} 
    for i in range(0, x): 
        client[i]={} 
        client[i]['social_security_number']= fake.ssn() 
        client[i]['first_name']= faker.first_name() 
        client[i]['last_name']= faker.last_name() 
        client[i]['gender']= faker.random_element(elements=('M', 'F')) 
        client[i]['date_of_birth']= faker.date() 
        client[i]['created_date'] = datetime.datetime.now()
        client[i]['citizenship'] = datetime.datetime.now()
        client[i]['is_deleted'] = 'false'

    
    client_df = pd.DataFrame.from_dict(client)
    client_f = client_df.T
    client_f['social_security_number'] = client_f['social_security_number']
    Client = client_f[['citizenship','first_name','last_name','social_security_number'
                ,'gender','date_of_birth','created_date','is_deleted']]
    return Client

In [5]:
Client = create_clientdata(1000)
tuples = [tuple(x) for x in Client.to_numpy()]
    # Comma-separated dataframe columns
cols = ','.join(list(Client.columns))
query  = "INSERT INTO %s(%s) VALUES %%s" % ('Client', cols)
cursor = conn.cursor()
psycopg2.extras.execute_values(cursor, query, tuples)
conn.commit()


#Load Data into Contact details for each Client (1 per client)

In [6]:
def create_contactdata(x): 
  
    clients  = pd.read_sql("select client_id,first_name,last_name from client", conn);
    contacts ={} 
    for i in range(0, x): 
        contacts[i]={} 
        contacts[i]['phone_number']= fake.phone_number()
        contacts[i]['id']= i +1
        contacts[i]['created_date'] = datetime.datetime.now()
        contacts[i]['is_deleted'] = 'false'
        df = pd.DataFrame.from_dict(contacts)
        contact = df.T
    clients.insert(0, 'id', range(1, 1 + len(clients)))
    contact_details = pd.merge(contact,clients, on=['id','id'])
    contact_details['email_id'] = contact_details['first_name'].str.lower()+'.'+contact_details['last_name'].str.lower()+'@gmail.com'
    Contacts = contact_details[['client_id','phone_number','email_id','created_date','is_deleted']]
    return Contacts

In [7]:
Contact_Details = create_contactdata(1000)
tuples = [tuple(x) for x in Contact_Details.to_numpy()]
    # Comma-separated dataframe columns
cols = ','.join(list(Contact_Details.columns))
query  = "INSERT INTO %s(%s) VALUES %%s" % ('contact_details', cols)
cursor = conn.cursor()
psycopg2.extras.execute_values(cursor, query, tuples)
conn.commit()

#Load Data into family details for each Client (1 per client)

In [8]:
def create_familydata(x): 
  
    clients  = pd.read_sql("select client_id,first_name,last_name from client", conn);
    family ={} 
    for i in range(0, x): 
        family[i]={} 
        family[i]['marital_status']= faker.random_element(elements=('Married', 'Single','Divorced','Engaged','In Civil Union'))
        family[i]['residential_status']= faker.random_element(elements=('Alone', 'With Family','Shared Flat'))
        family[i]['no_of_dependents']=  faker.pyint(min_value=0, max_value=3, step=1) 
        family[i]['id']= i +1
        family[i]['created_date'] = datetime.datetime.now()
        family[i]['is_deleted'] = 'false'
        df = pd.DataFrame.from_dict(family)
        fam = df.T
    clients.insert(0, 'id', range(1, 1 + len(fam)))
    families = pd.merge(fam,clients, on=['id','id'])
    Family = families[['client_id','marital_status','residential_status','no_of_dependents','created_date','is_deleted']]
    return Family

In [9]:
Family_Details = create_familydata(1000)

tuples = [tuple(x) for x in Family_Details.to_numpy()]
    # Comma-separated dataframe columns
cols = ','.join(list(Family_Details.columns))
query  = "INSERT INTO %s(%s) VALUES %%s" % ('family_details', cols)
cursor = conn.cursor()
psycopg2.extras.execute_values(cursor, query, tuples)
conn.commit()

#Load Data for each credit application

In [13]:

def create_credit_application(x): 
    clients  = pd.read_sql("select client_id,first_name,last_name from client", conn);
    credit ={} 
    for i in range(0, x): 
        credit[i]={} 
        credit[i]['status_date']= faker.date_between(start_date='-3y', end_date='today')
        credit[i]['is_rejected_before']= faker.boolean()
        credit[i]['status']= faker.random_element(elements=('In-progress', 'Rejected','New','Approved','On-hold'))
        credit[i]['status_comments']= faker.sentence(nb_words=6, variable_nb_words=True, ext_word_list=None)
        credit[i]['created_date'] = datetime.datetime.now()
        credit[i]['is_deleted'] = 'false'
        credit[i]['id']= i +1
        df = pd.DataFrame.from_dict(credit)
        cred = df.T
    clients.insert(0, 'id', range(1, 1 + len(clients)))
    credit_app = pd.merge(cred,clients, on=['id','id'])
    CAS = credit_app[['client_id','status_date','is_rejected_before','status','status_comments','created_date','is_deleted']]
    return CAS




In [14]:
Credit_Application = create_credit_application(1000)
tuples = [tuple(x) for x in Credit_Application.to_numpy()]
    # Comma-separated dataframe columns
cols = ','.join(list(Credit_Application.columns))
query  = "INSERT INTO %s(%s) VALUES %%s" % ('credit_application_status', cols)
cursor = conn.cursor()
psycopg2.extras.execute_values(cursor, query, tuples)
conn.commit()

#Load Data for address for each client

In [15]:

def create_address(x): 
    clients  = pd.read_sql("select client_id,first_name,last_name from client", conn);
    address ={} 
    for i in range(0, x): 
        address[i]={} 
        address[i]['house_no']= faker.building_number()
        address[i]['street_name']= faker.street_address()
        address[i]['city']= faker.random_element(elements=('Vienna','Graz','Linz','Salzburg','Innsbruck','St. Polten')) 
        address[i]['postal_code']= faker.random_element(elements=('1010','1200','2020','2010','3010','4010')) 
        address[i]['created_date'] = datetime.datetime.now()
        address[i]['is_deleted'] = 'false'
        address[i]['id']= i +1
        df = pd.DataFrame.from_dict(address)
        adds = df.T
    clients.insert(0, 'id', range(1, 1 + len(clients)))
    address_details = pd.merge(adds,clients, on=['id','id'])
    Address = address_details[['client_id','house_no','street_name','city','postal_code','created_date','is_deleted']]
    return Address

In [16]:
Address_details = create_address(1000)
tuples = [tuple(x) for x in Address_details.to_numpy()]
    # Comma-separated dataframe columns
cols = ','.join(list(Address_details.columns))
query  = "INSERT INTO %s(%s) VALUES %%s" % ('address_details', cols)
cursor = conn.cursor()
psycopg2.extras.execute_values(cursor, query, tuples)
conn.commit()

#Load Data for each client, 1 ID, 1 Meldezettle, 3 payslips

In [17]:
    clients  = pd.read_sql("select client_id,first_name,last_name from client", conn);
    Document_Columns = pd.DataFrame( columns = ['Name', 'Description','location','date','status']) 
    data = {
        'name':['ID','Payslip1','Payslip2','Payslip3','Meldezettle'],
        'description':['Identity Card','Payslip for month 1','Payslip for month 2','Payslip for month 3','Address proff'],
        'location_url':['s3://core-banking-sample-data/Document samples/Id.jpg'
              ,'s3://core-banking-sample-data/Document samples/Payslip.png'
              ,'s3://core-banking-sample-data/Document samples/Payslip.png'
              ,'s3://core-banking-sample-data/Document samples/Payslip.png'
              ,'s3://core-banking-sample-data/Document samples/meldezettle.png'],
        'created_date':[datetime.datetime.now(),datetime.datetime.now(),datetime.datetime.now(),datetime.datetime.now(),datetime.datetime.now()],
        'is_deleted':['false','false','false','false','false'],
        'category':['ID','Payslip','Payslip','Payslip','Meldezettle']
       
    }
    Documents = pd.DataFrame(data, columns = ['name', 'description','location_url','created_date','is_deleted','category'])
    Documents['id'] = 1
    clients['id'] = 1
    Documents = pd.merge(Documents, clients, on=['id'])
    Documents = Documents[['client_id','name', 'description','location_url','created_date','is_deleted','category']]
    #Documents    

In [18]:
tuples = [tuple(x) for x in Documents.to_numpy()]
    # Comma-separated dataframe columns
cols = ','.join(list(Documents.columns))
query  = "INSERT INTO %s(%s) VALUES %%s" % ('document_list', cols)
cursor = conn.cursor()
psycopg2.extras.execute_values(cursor, query, tuples)
conn.commit()

#Load Data for each clients financial details

In [1]:

def create_financial_details(x): 
  
    clients  = pd.read_sql("select client_id,first_name,last_name from client", conn);
    credit ={} 
    for i in range(0, x): 
        credit[i]={} 
        credit[i]['employment_status']= faker.random_element(elements=('Unemployed','Employed-Fulltime','Employed-Parttime','Student','Selfemployed')) 

        credit[i]['id']= i +1
        credit[i]['created_date'] = datetime.datetime.now()
        credit[i]['is_deleted'] = 'false'
        df = pd.DataFrame.from_dict(credit)
        fin = df.T
    
    fin.loc[fin['employment_status'] == 'Unemployed', 'annual_income'] = 0
    fin.loc[fin['employment_status'] == 'Student', 'annual_income'] = faker.pyint(min_value=10000, max_value=25000, step=1000) 
    fin.loc[fin['employment_status'] == 'Employed-Fulltime', 'annual_income'] = faker.pyint(min_value=30000, max_value=80000, step=1000) 
    fin.loc[fin['employment_status'] == 'Employed-Parttime', 'annual_income']  = faker.pyint(min_value=10000, max_value=40000, step=1000)
    fin.loc[fin['employment_status'] == 'Selfemployed', 'annual_income']  = faker.pyint(min_value=10000, max_value=90000, step=1000)

    fin.loc[(fin['employment_status'] == 'Unemployed')| (fin['employment_status'] == 'Student'), 'employed_years'] = 0
    fin.loc[(fin['employment_status'] == 'Employed-Fulltime')| (fin['employment_status'] == 'Selfemployed'), 'employed_years'] = faker.pyint(min_value=1, max_value=20, step=1) 
    fin.loc[fin['employment_status'] == 'Employed-Parttime', 'employed_years']  = faker.pyint(min_value=1, max_value=5, step=1) 

    fin.loc[cred['employment_status'] == 'Employed-Fulltime', 'position'] = faker.random_element(elements=('Staff','Management','Owner','Selfemployed'))  
    fin.loc[cred['employment_status'] == 'Selfemployed', 'position'] = 'Selfemployed' 
    fin.loc[(cred['employment_status'] == 'Employed-Parttime')|(fin['employment_status'] == 'Student'), 'position']  = faker.random_element(elements=('Parttime','Internship')) 
    fin.loc[cred['employment_status'] == 'Unemployed', 'position'] = 'N/A'

    fin.loc[(cred['employment_status'] == 'Employed-Fulltime')|(fin['employment_status'] == 'Employed-Parttime'), 'company_employed'] = faker.random_element(elements=('Public','Private')) 
    fin.loc[(cred['employment_status'] != 'Employed-Fulltime')&(fin['employment_status'] != 'Employed-Parttime'), 'company_employed'] = 'N/A'
    #credit[i]['company_employed']= faker.pyint(min_value=0, max_value=200, step=1)
    
    
    clients.insert(0, 'id', range(1, 1 + len(clients)))
    financial_details = pd.merge(fin,clients, on=['id','id'])
    
    #Credit = credit_info[['client_id','credit_amount','interest_rate','planned_emi','duration_in_months','created_date','is_deleted']]
    #return Credit
    financial_details = financial_details[['client_id','employment_status','annual_income','position','company_employed','employed_years','created_date','is_deleted']]

    return financial_details

In [20]:
financial_details = create_financial_details(1000)
tuples = [tuple(x) for x in financial_details.to_numpy()]
    # Comma-separated dataframe columns
cols = ','.join(list(financial_details.columns))
query  = "INSERT INTO %s(%s) VALUES %%s" % ('financial_information', cols)
cursor = conn.cursor()
psycopg2.extras.execute_values(cursor, query, tuples)
conn.commit()

#Load Data for each clients credit details for each loan

In [23]:
def create_credit_details(x):
    clients  = pd.read_sql("select client_id,first_name,last_name from client", conn);
    credit ={} 
    for i in range(0, x): 
        credit[i]={} 
        credit[i]['credit_amount']= faker.pyint(min_value=1, max_value=200, step=1)
        credit[i]['id']= i +1
        credit[i]['created_date'] = datetime.datetime.now()
        credit[i]['is_deleted'] = 'false'
        df = pd.DataFrame.from_dict(credit)
        cred = df.T

    cred['credit_amount'] = cred['credit_amount']*500
    #Interest
    cred.loc[cred["credit_amount"] <=5000 , 'interest_rate'] = round(random.uniform(2, 3),2)
    cred.loc[(cred["credit_amount"] <=10000) & (cred["credit_amount"] > 5000), 'interest_rate'] = round(random.uniform(1.5, 2),2)
    cred.loc[(cred["credit_amount"] <=50000) & (cred["credit_amount"] > 10000)  , 'interest_rate'] = round(random.uniform(1, 1.5),2)
    cred.loc[(cred["credit_amount"] <=100000) & (cred["credit_amount"] > 50000), 'interest_rate']  = round(random.uniform(0.5, 1),2)
    
     #Duration Months
    cred.loc[cred["credit_amount"] <=5000 , 'duration_in_months'] = 1
    cred.loc[(cred["credit_amount"] <=25000) & (cred["credit_amount"] > 5000), 'duration_in_months'] = random.randrange(1, 3)
    cred.loc[(cred["credit_amount"] <=100000) & (cred["credit_amount"] > 25000)  , 'duration_in_months'] = random.randrange(2, 8)

    cred["i"] =  ((cred["interest_rate"]/100)*cred["duration_in_months"])
    cred["x"] =  (cred["i"]+1).pow(cred["duration_in_months"].astype(int)*12)
    
    cred["planned_emi"] =  ((cred["i"]+1)*cred['credit_amount'])/(cred["duration_in_months"]*12)
    cred["duration_in_months"] = cred["duration_in_months"]*12
    clients.insert(0, 'id', range(1, 1 + len(clients)))
    Credit_details = pd.merge(cred,clients, on=['id','id'])
    Credit_details = Credit_details[['client_id','credit_amount','interest_rate','planned_emi','duration_in_months','created_date','is_deleted']]

    return Credit_details
    


In [24]:
Credit_details = create_credit_details(1000)
tuples = [tuple(x) for x in Credit_details.to_numpy()]
    # Comma-separated dataframe columns
cols = ','.join(list(Credit_details.columns))
query  = "INSERT INTO %s(%s) VALUES %%s" % ('credit_information', cols)
cursor = conn.cursor()
psycopg2.extras.execute_values(cursor, query, tuples)
conn.commit()